konfigurasi.py

In [ ]:
# konfigurasi.py
import os

BASE_DIR = os.path.dirname(os.path.abspath(__file__)) 
NAMA_DB = 'pengeluaran_harian.db'
DB_PATH = os.path.join(BASE_DIR, NAMA_DB)
KATEGORI_PENGELUARAN = ["Makanan", "Transportasi", "Hiburan", "Tagihan", "Belanja", "Kesehatan", "Pendidikan", "Lainnya"]
KATEGORI_DEFAULT = "Lainnya"

 setup_db_pengeluaran.py

In [ ]:
# setup_db_pengeluaran.py import sqlite3
import sqlite3
import os
from konfigurasi import DB_PATH # Ambil path dari konfigurasi

def setup_database():
    print(f"Memeriksa/membuat database di: {DB_PATH}") 
    conn = None
    try:
        conn = sqlite3.connect(DB_PATH) 
        cursor = conn.cursor() 
        sql_create_table = """
        CREATE TABLE IF NOT EXISTS transaksi (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            deskripsi TEXT NOT NULL,
            jumlah REAL NOT NULL CHECK(jumlah > 0), kategori TEXT,
            tanggal DATE NOT NULL
        );"""
        print(" Membuat tabel 'transaksi' (jika belum ada)...") 
        cursor.execute(sql_create_table)
        conn.commit(); print(" -> Tabel 'transaksi' siap.") 
        return True
    except sqlite3.Error as e: 
        print(f" -> Error SQLite saat setup: {e}"); 
        return False
    finally:
        if conn: conn.close(); print(" -> Koneksi DB setup ditutup.")

if __name__ == "  main  ":
    print("--- Memulai Setup Database Pengeluaran ---")
    if setup_database():	
        print(f"\nSetup	database '{os.path.basename(DB_PATH)}' selesai.")
    else: print(f"\nSetup database GAGAL.") 
    print("--- Setup Database Selesai ---")

database.py

In [ ]:
# database.py
import sqlite3
import pandas as pd
from konfigurasi import DB_PATH

def get_db_connection() -> sqlite3.Connection | None:
    """Membuka dan mengembalikan koneksi baru ke database SQLite."""
    try:
        conn = sqlite3.connect(DB_PATH, timeout=10, detect_types=sqlite3.PARSE_DECLTYPES)
        conn.row_factory = sqlite3.Row # Akses kolom by name
        return conn
    except sqlite3.Error as e: print(f"ERROR [database.py] Koneksi DB gagal: {e}"); return None

def execute_query(query: str, params: tuple | None = None):
    conn = get_db_connection()
    if not conn:
        return None
    try:
        cursor = conn.cursor()
        cursor.execute(query, params or [])
        conn.commit()
        return cursor.rowcount  # ← jumlah baris yang terpengaruh
    except sqlite3.Error as e:
        print(f"ERROR [database.py] Query gagal: {e}")
        conn.rollback()
        return None
    finally:
        conn.close()


def fetch_query(query: str, params: tuple = None, fetch_all: bool = True):
    """Menjalankan query SELECT dan mengembalikan hasil."""
    conn = get_db_connection();
    if not conn: return None
    try:
        cursor = conn.cursor();
        if params: cursor.execute(query, params)
        else: cursor.execute(query)
        result = cursor.fetchall() if fetch_all else cursor.fetchone();
        return result
    except sqlite3.Error as e: print(f"ERROR [database.py] Fetch gagal: {e} | Query: {query[:60]}"); return None
    finally:
        if conn: conn.close()

def get_dataframe(query: str, params: tuple | None = None) -> pd.DataFrame:
    """Menjalankan query SELECT dan mengembalikan DataFrame Pandas."""
    conn = get_db_connection();
    if not conn: return pd.DataFrame()
    try: df = pd.read_sql_query(query, conn, params=params); return df
    except Exception as e: print(f"ERROR [database.py] Gagal baca ke DataFrame: {e}"); return pd.DataFrame()
    finally:
        if conn: conn.close()

def setup_database_initial(): # Fungsi setup dipindah ke sini juga (opsional)
    """Memastikan tabel transaksi ada (dipanggil oleh AnggaranHarian jika perlu)."""
    print(f"Memeriksa/membuat tabel di database (via database.py): {DB_PATH}")
    conn = get_db_connection();
    if not conn: return False
    try:
        cursor = conn.cursor();
        sql_create_table = """
        CREATE TABLE IF NOT EXISTS transaksi (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            deskripsi TEXT NOT NULL,
            jumlah REAL NOT NULL CHECK (jumlah > 0),
            kategori TEXT,
            tanggal DATE NOT NULL
        );"""
        cursor.execute(sql_create_table); conn.commit(); print(" -> Tabel 'transaksi' siap."); return True
    except sqlite3.Error as e: print(f"Error SQLite saat setup tabel: {e}"); return False
    finally:
        if conn: conn.close()

model.py

In [ ]:
# model.py
import datetime

class Transaksi:
    """Merepresentasikan satu entitas transaksi pengeluaran (Data Class)."""
    def __init__(self, deskripsi: str, jumlah: float, kategori: str,
                 tanggal: datetime.date | str, id_transaksi: int | None = None):
        self.id = id_transaksi
        self.deskripsi = str(deskripsi) if deskripsi else "Tanpa Deskripsi"
        try:
            jumlah_float = float(jumlah)
            self.jumlah = jumlah_float if jumlah_float > 0 else 0.0
            if jumlah_float <= 0: print(f"Peringatan: Jumlah '{jumlah}' harus positif.")
        except (ValueError, TypeError): self.jumlah = 0.0; print(f"Peringatan: Jumlah '{jumlah}' tidak valid.")
        self.kategori = str(kategori) if kategori else "Lainnya"
        if isinstance(tanggal, datetime.date): self.tanggal = tanggal
        elif isinstance(tanggal, str):
            try: self.tanggal = datetime.datetime.strptime(tanggal, "%Y-%m-%d").date()
            except ValueError: self.tanggal = datetime.date.today(); print(f"Peringatan: Format tgl '{tanggal}' salah.")
        else: self.tanggal = datetime.date.today(); print(f"Peringatan: Tipe tgl '{type(tanggal)}' tidak valid.")

    def __repr__(self) -> str:
        try:
            import locale; locale.setlocale(locale.LC_ALL, 'id_ID.UTF-8')
            jml_str = locale.format_string("%.0f", self.jumlah, grouping=True)
        except: jml_str = f"{self.jumlah:.0f}"
        return f"Transaksi (ID: {self.id}, Tgl:{self.tanggal.strftime('%Y-%m-%d')}), Jml:{jml_str}, Kat:'{self.kategori}', Desc:'{self.deskripsi}')"

    def to_dict(self) -> dict:
        return {"deskripsi": self.deskripsi, "jumlah": self.jumlah,
                "kategori": self.kategori, "tanggal": self.tanggal.strftime("%Y-%m-%d")}

manajer_anggaran.py

In [ ]:
import datetime
import pandas as pd
import database  # Import your database module
from model import Transaksi

class AnggaranHarian:
    """Mengelola logika bisnis pengeluaran harian (Repository Pattern)."""
    
    _db_setup_done = False

    def __init__(self):  # perbaikan di sini
        if not AnggaranHarian._db_setup_done:
            print("[AnggaranHarian] Melakukan pengecekan/setup database awal...")
            if database.setup_database_initial():
                AnggaranHarian._db_setup_done = True
                print("[AnggaranHarian] Database siap.")
            else:
                print("[AnggaranHarian] KRITICAL: Setup database awal GAGAL!")

    def tambah_transaksi(self, transaksi) -> bool:
        if not isinstance(transaksi, Transaksi) or transaksi.jumlah <= 0:
            return False
        sql = "INSERT INTO transaksi (deskripsi, jumlah, kategori, tanggal) VALUES (?, ?, ?, ?)"
        params = (transaksi.deskripsi, transaksi.jumlah, transaksi.kategori, transaksi.tanggal.strftime("%Y-%m-%d"))
        last_id = database.execute_query(sql, params)
        if last_id is not None:
            transaksi.id = last_id
            return True
        return False

    def get_dataframe_transaksi(self, filter_tanggal: datetime.date | None = None) -> pd.DataFrame:
        query = "SELECT id, tanggal, kategori, deskripsi, jumlah FROM transaksi"
        params = None
        if filter_tanggal:
            query += " WHERE tanggal = ?"
            params = (filter_tanggal.strftime("%Y-%m-%d"),)
        query += " ORDER BY tanggal DESC, id DESC"
        df = database.get_dataframe(query, params=params)
    
        if not df.empty:
            try:
                import locale
                locale.setlocale(locale.LC_ALL, 'id_ID.UTF-8')
                df['Jumlah (Rp)'] = df['jumlah'].map(lambda x: locale.currency(x or 0, grouping=True, symbol='Rp ')[:-3])
            except:
                df['Jumlah (Rp)'] = df['jumlah'].map(lambda x: f"Rp {x or 0:,.0f}".replace(",", "."))
                df = df[['id', 'tanggal', 'kategori', 'deskripsi', 'Jumlah (Rp)']]  # Pastikan ID ada di sini
        return df

    def hapus_transaksi(self, id_transaksi: int) -> bool:
        sql = "DELETE FROM transaksi WHERE id = ?"
        result = database.execute_query(sql, (id_transaksi,))
        return result > 0

    def hitung_total_pengeluaran(self, tanggal=None):
        query = "SELECT SUM(jumlah) as total FROM transaksi"
        params = ()

        if tanggal:
            query += " WHERE tanggal = ?"
            params = (tanggal,)

        result = database.fetch_query(query, params, fetch_all=False)  # ← BENAR

        if result:
            return float(result['total']) if result['total'] else 0.0
        return 0.0

    def get_pengeluaran_per_kategori(self, tanggal=None) -> dict:
        """Mengembalikan pengeluaran total per kategori dalam bentuk dictionary."""
        query = "SELECT kategori, SUM(jumlah) as total FROM transaksi"
        params = ()
        if tanggal:
            query += " WHERE tanggal = ?"
            params = (tanggal,)
        query += " GROUP BY kategori"

        hasil = database.fetch_query(query, params=params, fetch_all=True)
        if not hasil:
            return {}
        return {row["kategori"] or "Tidak Dikategorikan": row["total"] for row in hasil}

main_app.py


In [ ]:
import streamlit as st
import datetime
import pandas as pd
from manajer_anggaran import AnggaranHarian  
from konfigurasi import KATEGORI_PENGELUARAN  
from model import Transaksi
import locale
import time

st.set_page_config(page_title="Catatan Pengeluaran", layout="wide", initial_sidebar_state="expanded")

def format_rp(angka):
    try: 
        return locale.currency(angka or 0, grouping=True, symbol='Rp ')[:-3]
    except: 
        return f"Rp {angka or 0:,.0f}".replace(",", ".")
    
@st.cache_resource
def get_anggaran_manager():
    print(">>> STREAMLIT: (Cache Resource) Menginisialisasi AnggaranHarian...")
    return AnggaranHarian()  
anggaran = get_anggaran_manager()


def halaman_input(anggaran: AnggaranHarian):
    st.header("Tambah Pengeluaran Baru")
    with st.form("form_transaksi_baru", clear_on_submit=True):
        col1, col2 = st.columns([3, 1])
        with col1: 
            deskripsi = st.text_input("Deskripsi*", placeholder="Contoh: Makan siang")
        with col2: 
            kategori = st.selectbox("Kategori*:", KATEGORI_PENGELUARAN, index=0)
        col3, col4 = st.columns([1, 1])
        with col3: 
            jumlah = st.number_input("Jumlah (Rp)*:", min_value=0.01, step=1000.0, format="%.0f", value=None, placeholder="Contoh: 25000")
        with col4: 
            tanggal = st.date_input("Tanggal*:", value=datetime.date.today())
        submitted = st.form_submit_button(" Simpan Transaksi")
        
        if submitted:
            if not deskripsi: 
                st.warning("Deskripsi wajib!", icon="⚠")
            elif jumlah is None or jumlah <= 0: 
                st.warning("Jumlah wajib!", icon="⚠")
            else:
                with st.spinner("Menyimpan..."):
                    tx = Transaksi(deskripsi, float(jumlah), kategori, tanggal)
                    if anggaran.tambah_transaksi(tx): 
                        st.success(f"OK! Simpan.", icon="✅")
                        st.cache_data.clear()
                        st.rerun()
                    else: 
                        st.error("Gagal simpan.", icon="❌")

def halaman_riwayat(anggaran: AnggaranHarian):
    st.subheader("Detail Semua Transaksi")
    if st.button("Refresh Riwayat"): 
        st.cache_data.clear()
        st.rerun()
    
    with st.spinner("Memuat riwayat..."): 
        df_transaksi = anggaran.get_dataframe_transaksi()
    
    if df_transaksi is None: 
        st.error("Gagal ambil riwayat.")
    elif df_transaksi.empty: 
        st.info("Belum ada transaksi.")
    else:
        st.dataframe(df_transaksi, use_container_width=True, hide_index=True)

    # Input ID untuk hapus transaksi
    st.subheader("Hapus Transaksi")
    id_transaksi = st.number_input("ID Transaksi yang ingin dihapus:", min_value=1, step=1)
    confirm_button = st.button("Konfirmasi Hapus")

    if confirm_button:
        if id_transaksi:
            # Hapus transaksi
            with st.spinner("Menghapus..."):
                if anggaran.hapus_transaksi(id_transaksi):
                    st.success(f"Transaksi dengan ID {id_transaksi} berhasil dihapus.", icon="✅")
                    st.cache_data.clear()
                    time.sleep(1.5) 
                    st.rerun()
                else:
                    st.error(f"Gagal menghapus transaksi dengan ID {id_transaksi}. Pastikan ID valid.", icon="❌")
        else:
            st.warning("Masukkan ID transaksi yang valid.")


def halaman_ringkasan(anggaran: AnggaranHarian):
    st.subheader("Ringkasan Pengeluaran")
    col_filter1, col_filter2 = st.columns([1, 2])
    with col_filter1:
        pilihan_periode = st.selectbox("Filter Periode:", ["Semua Waktu", "Hari Ini", "Pilih Tanggal"], key="filter_periode", on_change=lambda: st.cache_data.clear())
    tanggal_filter = None
    label_periode = "(Semua Waktu)"
    if pilihan_periode == "Hari Ini":
        tanggal_filter = datetime.date.today()
        label_periode = f"({tanggal_filter.strftime('%d %b')})"
    elif pilihan_periode == "Pilih Tanggal Tertentu":
        if 'tanggal_pilihan_state' not in st.session_state:
            st.session_state.tanggal_pilihan_state = datetime.date.today()
        tanggal_filter = st.date_input("Pilih Tanggal:", value=st.session_state.tanggal_pilihan_state, key="tanggal_pilihan", on_change=lambda: setattr(st.session_state, 'tanggal_pilihan_state', st.session_state.tanggal_pilihan) or st.cache_data.clear())
        label_periode = f"({tanggal_filter.strftime('%d %b %Y')})"
    
    with col_filter2:
        @st.cache_data(ttl=300)  
        def hitung_total_cached(tgl_filter): 
            return anggaran.hitung_total_pengeluaran(tanggal=tgl_filter)
        
        total_pengeluaran = hitung_total_cached(tanggal_filter)
        st.metric(label=f"Total Pengeluaran {label_periode}", value=format_rp(total_pengeluaran))

    st.divider()
    st.subheader(f"Pengeluaran per Kategori {label_periode}")
    @st.cache_data(ttl=300)  
    def get_kategori_cached(tgl_filter): 
        return anggaran.get_pengeluaran_per_kategori(tanggal=tgl_filter)

    with st.spinner(f"Memuat ringkasan kategori..."): 
        dict_per_kategori = get_kategori_cached(tanggal_filter)

    if not dict_per_kategori: 
        st.info(f"Tidak ada data untuk periode ini.")
    else:
        try:
            data_kategori = [{"Kategori": kat, "Total": jml} for kat, jml in dict_per_kategori.items()]
            df_kategori = pd.DataFrame(data_kategori).sort_values(by="Total", ascending=False).reset_index(drop=True)
            df_kategori['Total (Rp)'] = df_kategori['Total'].apply(format_rp)
            col_kat1, col_kat2 = st.columns(2)
            with col_kat1: 
                st.write("Tabel:")
                st.dataframe(df_kategori[['Kategori', 'Total (Rp)']], hide_index=True, use_container_width=True)
            with col_kat2: 
                st.write("Grafik:")
                st.bar_chart(df_kategori.set_index('Kategori')['Total'], use_container_width=True)
        except Exception as e: 
            st.error(f"Gagal tampilkan ringkasan: {e}")


def main():
    st.sidebar.title("Catatan Pengeluaran")
    menu_pilihan = st.sidebar.radio("Pilih Menu:", ["Tambah", "Riwayat", "Ringkasan"], key="menu_utama")
    st.sidebar.markdown("---")
    st.sidebar.info("Jobsheet - Aplikasi Keuangan")
    manajer_anggaran = get_anggaran_manager()
    
    if menu_pilihan == "Tambah": 
        halaman_input(manajer_anggaran)
    elif menu_pilihan == "Riwayat": 
        halaman_riwayat(manajer_anggaran)
    elif menu_pilihan == "Ringkasan":
        halaman_ringkasan(manajer_anggaran)
    
    st.markdown("---")
    st.caption("Pengembangan Aplikasi Berbasis OOP")

if __name__== "__main__":
    main() 